In [84]:
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats, leaguegamefinder, commonteamroster, leagueseasonmatchups
from nba_api.stats.static import players, teams
from os import path
import pandas as pd
import pickle
import numpy as np

In [85]:
""" for saving and loading things to a pickle file. Don't need to add extension to the file name """
def save_obj(obj, filename, dirname='pickle_files', ):
    with open(path.join(dirname, filename + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(filename, dirname='pickle_files'):
    with open(path.join(dirname, filename + '.pkl'), 'rb') as f:
        return pickle.load(f)

In [86]:
teams_df = pd.DataFrame(teams.get_teams())
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [87]:
team_ids = teams_df.loc[:, ("id","abbreviation")]

In [88]:
team_ids["abbreviation"]

0     ATL
1     BOS
2     CLE
3     NOP
4     CHI
5     DAL
6     DEN
7     GSW
8     HOU
9     LAC
10    LAL
11    MIA
12    MIL
13    MIN
14    BKN
15    NYK
16    ORL
17    IND
18    PHI
19    PHX
20    POR
21    SAC
22    SAS
23    OKC
24    TOR
25    UTA
26    MEM
27    WAS
28    DET
29    CHA
Name: abbreviation, dtype: object

In [89]:
""" load the team_names dictionary in pickle_files """
team_names = load_obj('team_names')

# Normalizing 

In [90]:
# seasons = list(range(22009, 22019))
# season_avgs = {}
# for s in seasons:
#     team_averages = np.zeros(19)
#     for i in team_ids["id"]:
#         teams_df = get_season_team(s, i, team_names_test)
#         team_averages += get_team_avgs(teams_df, num_rows(teams_df), num_rows(teams_df) - 1)
#     season_avgs[s] = team_averages / len(team_ids["id"])

In [91]:
df = get_season_team(22018, 1610612737, team_names_test)
df = df.drop(df.columns[list(range(0, 9))], axis = 1).values.tolist()

In [92]:
from sklearn.preprocessing import StandardScaler

In [93]:
def normalize(prev_season):
    scaler = StandardScaler()
    season = []
    for i in team_ids["id"]:
        teams_df = get_season_team(prev_season, i, team_names_test)
        teams_list = teams_df.drop(teams_df.columns[list(range(0, 9))], axis = 1).values.tolist()
        season += teams_list
    
    scaler.fit(season)
    return scaler

In [94]:
# *** should only have to run the loading cell below ***
# get team info into dataframes
# store dictionary key = team abbreviation, value = df with game data
# import time

# team_names = {}
# for i in range(len(team_ids)): 
#     temp_id = team_ids["id"][i]
#     temp_team_name = team_ids["abbreviation"][i]
#     print('getting team: {0}...'.format(temp_team_name))
#     team_names[temp_team_name] = leaguegamefinder.LeagueGameFinder(team_id_nullable=temp_id).get_data_frames()[0]
#     time.sleep(1)

In [95]:
""" save dictionary to a file called team_names under pickle_files folder"""
# save_obj(team_names, 'team_names')

' save dictionary to a file called team_names under pickle_files folder'

# Messy Calculations

In [96]:
team_names_test = {}

ids = team_ids["id"]
abr = team_ids["abbreviation"]

for i in range(len(ids)):
    real_value = team_names[abr[i]]
    test_key = ids[i]
    team_names_test[test_key] = real_value

In [97]:
# for game in team_names['CHA'].values.tolist():
#     if (game[2] != "CHA"):
#         print(game[2])

all_abrs = {}
for team in team_names.keys():
    df = team_names[team]
    abrs = set()
    for game in df.values.tolist():
        abrs.add(game[2])
    for abr in abrs:
        all_abrs[abr] = df
#print(len(all_abrs))
all_abrs['ATL']

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612737,ATL,Atlanta Hawks,0021900105,2019-11-06,ATL vs. CHI,None,24,8,...,1.000,1,3,4,1,0.0,0,3,0,-9.0
1,22019,1610612737,ATL,Atlanta Hawks,0021900099,2019-11-05,ATL vs. SAS,W,240,108,...,0.688,9,39,48,22,9.0,5,18,20,8.0
2,22019,1610612737,ATL,Atlanta Hawks,0021900066,2019-10-31,ATL vs. MIA,L,238,97,...,0.692,16,34,50,20,5.0,5,16,15,-9.0
3,22019,1610612737,ATL,Atlanta Hawks,0021900052,2019-10-29,ATL @ MIA,L,240,97,...,0.667,9,24,33,28,14.0,7,20,29,-15.0
4,22019,1610612737,ATL,Atlanta Hawks,0021900043,2019-10-28,ATL vs. PHI,L,240,103,...,0.688,8,37,45,23,12.0,3,21,25,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,...,0.667,11,26,37,18,4.0,5,14,24,NaN
3248,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,...,0.692,19,27,46,31,14.0,13,18,27,NaN
3249,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,...,0.633,12,29,41,20,7.0,10,16,34,NaN
3250,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,...,0.633,27,21,48,28,14.0,7,23,35,NaN


In [98]:
""" gets the season games given the seasonID and abbreviation of the team """
""" Season is based on the year that the season STARTED in. I.E 2016-2017 has seasonID 2016"""
""" Reg season starts with 2, playoffs 4, pre 1 """
""" 
For some reason summer league and preseason games are sometimes included in reg season IDs.
The function accounts for this by only looking at Game IDs that start with 0021 for reg reason,
0041 for playoffs, 0011 for preseason
"""

def get_season_team(seasonID, team_id, team_names):
    seasonID = str(seasonID)
    gameID_prefix = seasonID[0]
    return team_names[team_id].loc[(team_names[team_id]["SEASON_ID"].str.contains(seasonID)) & \
                                    (team_names[team_id]['GAME_ID'].str.startswith('00{0}'.format(gameID_prefix)))]


""" gets the number of rows in the passed df"""
def num_rows(df):
    return len(df.index)

""" returns the number of features of the df"""
def num_features(df):
    return len(df.columns)

""" returns a list of the features from the start index moving forward. Using 9 for now """
def get_features(df, start_idx=9):
    return [feature for feature in df.columns.tolist()[start_idx:]]

""" 
util func: gets last n games given the curr game
in the df with the correct season, year
"""
def last_n_games(df, n, curr_game):
    n_rows = num_rows(df)
    # the first row is the latest game. I.E for reg season it's 82nd game
    
"""
returns avg team stats vector over the last n games given
the current game. Assumes games 0 indexed
"""
def get_team_avgs(season_games, curr_game, n):
    total_games = num_rows(season_games)
    assert curr_game > n, "curr game ({0}) > n ({1})".format(curr_game, n)
    assert total_games > n, "n ({0}) is more than the total number of games ({1})".format(n, total_games)
    # the real index of the current game
    game_idx = total_games - curr_game - 1
    # start at 9th index to get meaningful stats
    start_idx = 9
    team_avgs = np.zeros(num_features(season_games) - start_idx)
    for i in range(game_idx - n, game_idx):
        game_stats = np.array(season_games.iloc[i].tolist()[start_idx:])
        team_avgs += game_stats
    return team_avgs / n

def print_team_avgs(season_games, team_avgs):
    features = get_features(season_games)
    for stat, feature in zip(team_avgs, features):
        print("{0}:{1}".format(feature, stat))

In [99]:
""" 
get the 2018 season games for the Atlanta Hawks. 
"""
atl_reg_season = get_season_team(22018, 1610612737, team_names_test)
"""
print the avgs of the 10 games before the 16th game that the Hawks played in the 2017-2018 season
"""
atl_team_avgs = get_team_avgs(atl_reg_season, 16, 10)
#print_team_avgs(atl_reg_season, atl_team_avgs)

In [100]:
def find_opponent_id(opp_abr, season_id):
    
    return all_abrs[opp_abr].loc[all_abrs[opp_abr]["SEASON_ID"].str.contains(str(season_id))]["TEAM_ID"].iloc[0]

In [101]:
def find_opponent(n, curr_team, season_id):
    full_df = get_season_team(season_id, curr_team, team_names_test).reset_index()
    
    df_of_game = full_df.iloc[[-n]]
    opp_abr = df_of_game["MATCHUP"].values[0][-3::]
    return find_opponent_id(opp_abr, season_id)

In [102]:
"""get the opponent of the Atlanta Hawks's 2nd game of the the 2018 season"""

find_opponent(2, 1610612737, 22018)

1610612763

In [103]:
def convert_to_01s(letter):
    return 1 if letter == "W" else 0

In [130]:
temp = get_season_team(22018, 1610612737, team_names_test)
tempstr = temp["MATCHUP"].iloc[num_rows(temp) - 15 - 1]
print(tempstr)
if "@" in  tempstr:
    print("@")
else:
    print("vs")

ATL @ IND
@


In [109]:
def check_home_team(team_df, curr_game):
    matchup = team_df["MATCHUP"].iloc[num_rows(team_df) - curr_game - 1]
    return False if "@" in matchup else True

In [149]:
"""
return the difference vector
"""
def find_diff_vector(curr_game, n, curr_team, season_id):
    
    #curr_team_season is a df
    curr_team_season = get_season_team(season_id, curr_team, team_names_test)
    curr_team_avgs = get_team_avgs(curr_team_season, curr_game, n)
    
        
    #check for home game
    if check_home_team(curr_team_season, curr_game):
    
    
        # find win/loss for single curr_game
        outcome = curr_team_season["WL"].iloc[num_rows(curr_team_season) - curr_game - 1]
        outcome = convert_to_01s(outcome)

        opponent = find_opponent(curr_game, curr_team, season_id)
        opponent_team_season = get_season_team(season_id, opponent, team_names_test)
        opponent_team_avgs = get_team_avgs(opponent_team_season, curr_game, n)

        # normalize 
        scaler = normalize(season_id - 1)
        curr_team_avgs_normalized = scaler.transform([curr_team_avgs])
        opp_team_avgs_normalized = scaler.transform([opponent_team_avgs])

        diff = np.subtract(curr_team_avgs_normalized, opp_team_avgs_normalized)

        return diff, outcome
    return None, None

In [137]:
a = [1,2,3]
a.append(None)
a

[1, 2, 3, None]

In [153]:
"""diff vector between Atlanta Hawks' stats and its opponents' stats on the Hawks' 16th game
based on the averages of the last 15 games before the nth game"""
find_diff_vector(15, 14, 1610612737, 22018)

(None, None)

In [273]:
# gets diff vectors for all the games 2-82 for given team and season
def get_season_diff_vectors(curr_team, season_id):
    diff_vector = []
    win_losses = []
    df_len = num_rows(get_season_team(season_id, curr_team, team_names_test))
    for i in range(2, df_len):
        diff, outcome = find_diff_vector(i, min(8, i-1), curr_team, season_id)
        if outcome != None:
            diff_vector.append(diff)
            win_losses.append(outcome)
    return diff_vector, win_losses

In [266]:
diffs, outcomes = get_season_diff_vectors(1610612737, 22012)
len(outcomes)

40

In [274]:
# get difference vectors for all teams for one season

def diff_vectors_per_season(season_id):
    team_per_season_dict = {}
    outcomes_per_season_dict = {}
    
    for team in team_ids["id"]:
        #print(team)
        diff, outcomes = get_season_diff_vectors(team, season_id)
        team_per_season_dict[team] = diff
        outcomes_per_season_dict[team] = outcomes
    
    return team_per_season_dict, outcomes_per_season_dict

In [160]:
diff_vectors_per_season(22018)

({1610612737: [array([[-1.36781631, -1.8638545 ,  0.21876701, -2.12511849,  0.70144075,
            2.09420744, -0.53528773,  0.        ,  0.48976617, -1.35094064,
           -0.27542302,  2.62707291,  2.03968045,  0.09593313, -0.17094554,
            1.59658496,  1.48071304, -1.66175246, -0.50690288]]),
   array([[-0.11053061, -0.19619521, -0.24307446,  0.00608045, -0.09352543,
            0.59834498, -0.58863256,  0.17140928,  0.51308837, -0.67785714,
           -0.36723069,  0.06254936, -0.1568985 ,  0.51164336,  0.22792738,
            0.79829248, -0.08974018,  0.87044176, -1.42415571]]),
   array([[ 0.88424489,  0.42508962,  0.60768614,  0.0456034 ,  0.37410174,
            1.27148309, -0.64749581,  0.85704641,  0.86292134,  0.04137156,
           -0.09180767, -0.43784549, -0.41839599,  0.92735359,  0.68378214,
            0.66524373, -0.08974018,  0.27695874,  0.12069116]]),
   array([[-0.37896209, -0.75675295, -0.39586412, -0.5446349 , -0.32065863,
            0.59834498, -0.840

In [275]:
# get diff vectors for all seasons for all teams

def ultimate_diff_vectors():
    ultimate_dict = {}
    ultimate_dict_outcomes = {}
    seasons = list(range(22009, 22019))

    for s in seasons:
        print(s)
        diff, outcomes = diff_vectors_per_season(s)
        ultimate_dict[s] = diff
        ultimate_dict_outcomes[s] = outcomes
        
    return ultimate_dict, ultimate_dict_outcomes

In [276]:
"""
key = season
value = dictionary with key as team id, values as difference numpy arrays
"""
ultimate_dict_8, ultimate_dict_8_outcomes = ultimate_diff_vectors()


22009
22010
22011
22012
22013
22014
22015
22016
22017
22018


In [18]:
# pickle the ultimate dicts PRE normalizing

#save_obj(ultimate_dict, 'ultimate_dict')
#ultimate_dict_pickled = load_obj('ultimate_dict')

#save_obj(ultimate_dict_outcomes, 'ultimate_dict_outcomes')
#ultimate_dict_outcomes_pickled = load_obj('ultimate_dict_outcomes')

In [11]:
# pickle the ultimate dicts POST normalizing

#save_obj(ultimate_dict_normalized, 'ultimate_dict_normalized')
#ultimate_dict_pickled = load_obj('ultimate_dict_normalized')

#save_obj(ultimate_dict_outcomes_normalized, 'ultimate_dict_outcomes_normalized')
#ultimate_dict_outcomes_pickled_normalized = load_obj('ultimate_dict_outcomes_normalized')

In [278]:
# pickle the ultimate dicts prev 8 games normalizing

save_obj(ultimate_dict_8, 'ultimate_dict_8')
ultimate_dict_pickled_8 = load_obj('ultimate_dict_8')

save_obj(ultimate_dict_8_outcomes, 'ultimate_dict_outcomes_8')
ultimate_dict_outcomes_pickled_8 = load_obj('ultimate_dict_outcomes_8')

In [163]:
# save_obj(ultimate_dict_homeOnly, 'ultimate_dict_homeOnly')
# ultimate_dict_homeOnly_pickled = load_obj('ultimate_dict_homeOnly')

# save_obj(ultimate_dict_homeOnly_outcomes, 'ultimate_dict_homeOnly_outcomes')
# ultimate_dict_homeOnly_outcomes_pickled = load_obj('ultimate_dict_homeOnly_outcomes')

## MODELS

In [279]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from pipeline import Model

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier

In [280]:
def massive_flatten(ultimate_lists):
    flattened_values = []
    for team in ultimate_lists.values():
        for games in team.values():
            flattened_values.extend(games)
    return flattened_values

In [281]:
def flatten_further(ult):
    for idx, vector in enumerate(ult):
        ult[idx] = vector.flatten()


In [282]:
X = massive_flatten(ultimate_dict_homeOnly_pickled)
y = massive_flatten(ultimate_dict_homeOnly_outcomes_pickled)
flatten_further(X)

In [283]:
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25)

In [284]:
m = Model(LogisticRegression(solver='lbfgs', multi_class='multinomial'))
m.train(Xtr, ytr)
preds = m.predict(Xte)

/anaconda3/envs/virtualenvir/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [285]:
accuracy = 1 - np.count_nonzero(yte - preds)/len(yte)
accuracy

0.5971418849948962

In [286]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(Xtr, ytr)
decision_tree.score(Xte, yte)

0.5559714188499489

In [287]:
random_tree = RandomForestClassifier()
random_tree.fit(Xtr, ytr)
random_tree.score(Xte, yte)

/anaconda3/envs/virtualenvir/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5706022456617897

In [288]:
from sklearn.neural_network import MLPClassifier

In [289]:
clf = MLPClassifier(solver='adam', activation='relu', alpha=5e-3,hidden_layer_sizes=(32,16),max_iter=3000, random_state=1)
clf.fit(Xtr, ytr)
nn_preds = clf.predict(Xte)
accuracy = 1 - np.count_nonzero(yte - nn_preds)/len(yte)
#accuracy
accuracy

0.5733242599523647